In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd

# Initialize the Dash App
app = dash.Dash(__name__)

us_stocks_df = pd.read_csv("ticker_correlation.csv")
us_stocks = us_stocks_df.iloc[:, 0].tolist()

# Initialize a dictionary to store stock data
stock_data = {}
company_names = {}

# Iterate over stock symbols
for symbol in us_stocks:
    ticker = yf.Ticker(symbol)
    data = ticker.history(start="2013-01-01", end=pd.Timestamp.today().strftime('%Y-%m-%d'))
    stock_data[symbol] = data
    company_names[symbol] = ticker.info.get('longName', 'N/A')

# Sort the company names alphabetically and create a mapping to their stock symbols
sorted_companies = sorted(company_names.items(), key=lambda x: x[1])
company_symbol_mapping = {name: symbol for symbol, name in sorted_companies}

def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    data['EMA_fast'] = data['Close'].ewm(span=fast_period, adjust=False).mean()
    data['EMA_slow'] = data['Close'].ewm(span=slow_period, adjust=False).mean()
    data['MACD'] = data['EMA_fast'] - data['EMA_slow']
    data['Signal_line'] = data['MACD'].ewm(span=signal_period, adjust=False).mean()
    return data

def calculate_stochastic_oscillator(data, k_period=14, d_period=3):
    data['L14'] = data['Low'].rolling(window=k_period).min()
    data['H14'] = data['High'].rolling(window=k_period).max()
    data['%K'] = 100 * ((data['Close'] - data['L14']) / (data['H14'] - data['L14']))
    data['%D'] = data['%K'].rolling(window=d_period).mean()
    return data

def calculate_bollinger_bands(data, window=20, num_std=2):
    data['Middle_Band'] = data['Close'].rolling(window=window).mean()
    data['STD'] = data['Close'].rolling(window=window).std()
    data['Upper_Band'] = data['Middle_Band'] + (data['STD'] * num_std)
    data['Lower_Band'] = data['Middle_Band'] - (data['STD'] * num_std)
    return data

# Dash layout
app.layout = html.Div([
    html.H1('Stock Trading Signals Dashboard'),
    dcc.Dropdown(
        id='company-selector',
        options=[{'label': name, 'value': name} for _, name in sorted_companies],
        value=sorted_companies[0][1]  # Default value
    ),
    dcc.Graph(id='trading-signals-plot'),
    dcc.Graph(id='rsi-plot'),
    dcc.Graph(id='MACD-plot'),
    dcc.Graph(id='stochastic-plot'),
    dcc.Graph(id='bollinger-plot')
])

# Callback to update the plots based on the selected company
@app.callback(
    [Output('trading-signals-plot', 'figure'),
     Output('rsi-plot', 'figure'),
     Output('MACD-plot', 'figure'),
     Output('stochastic-plot', 'figure'),
     Output('bollinger-plot', 'figure')
     ],
    [Input('company-selector', 'value')]
)

def update_plots(company_name):
    stock_symbol = company_symbol_mapping[company_name]
    df = stock_data[stock_symbol]
    # Calculate the Simple Moving Average (SMA), Exponential Moving Average (EMA), and RSI
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
    df['RSI'] = calculate_rsi(df)

    # Generate trading signals (1 = buy, -1 = sell)
    df['SMA_Signal'] = 0
    df['EMA_Signal'] = 0
    df.loc[df['SMA_50'] > df['SMA_200'], 'SMA_Signal'] = 1
    df.loc[df['SMA_50'] < df['SMA_200'], 'SMA_Signal'] = -1
    df.loc[df['EMA_50'] > df['EMA_200'], 'EMA_Signal'] = 1
    df.loc[df['EMA_50'] < df['EMA_200'], 'EMA_Signal'] = -1
    df['RSI_Signal'] = 0
    df.loc[df['RSI'] > 70, 'RSI_Signal'] = -1  # Overbought
    df.loc[df['RSI'] < 30, 'RSI_Signal'] = 1   # Oversold

    # Create the trading signals plot
    signals_fig = go.Figure()
    signals_fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Close Price', line=dict(color='blue')))
    signals_fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], name='50-Day SMA', line=dict(color='red')))
    signals_fig.add_trace(go.Scatter(x=df.index, y=df['SMA_200'], name='200-Day SMA', line=dict(color='orange')))
    signals_fig.add_trace(go.Scatter(x=df.index, y=df['EMA_50'], name='50-Day EMA', line=dict(color='green')))
    signals_fig.add_trace(go.Scatter(x=df.index, y=df['EMA_200'], name='200-Day EMA', line=dict(color='purple')))
    
    # Highlight buy and sell signals
    signals_fig.add_trace(go.Scatter(x=df[df['SMA_Signal'] == 1].index, y=df[df['SMA_Signal'] == 1]['SMA_50'],
                                     mode='markers', marker_symbol='triangle-up', marker_color='lime', marker_size=10, name='SMA Buy Signal'))
    signals_fig.add_trace(go.Scatter(x=df[df['SMA_Signal'] == -1].index, y=df[df['SMA_Signal'] == -1]['SMA_50'],
                                     mode='markers', marker_symbol='triangle-down', marker_color='crimson', marker_size=10, name='SMA Sell Signal'))
    signals_fig.add_trace(go.Scatter(x=df[df['EMA_Signal'] == 1].index, y=df[df['EMA_Signal'] == 1]['EMA_50'],
                                     mode='markers', marker_symbol='circle', marker_color='lime', marker_size=8, name='EMA Buy Signal'))
    signals_fig.add_trace(go.Scatter(x=df[df['EMA_Signal'] == -1].index, y=df[df['EMA_Signal'] == -1]['EMA_50'],
                                     mode='markers', marker_symbol='circle', marker_color='crimson', marker_size=8, name='EMA Sell Signal'))
    
    # Calculate Bollinger Bands and signals
    df = calculate_bollinger_bands(df)
    df['Bollinger_Signal'] = 0
    df.loc[df['Close'] < df['Lower_Band'], 'Bollinger_Signal'] = 1  # Buy signal
    df.loc[df['Close'] > df['Upper_Band'], 'Bollinger_Signal'] = -1 # Sell signal
    
    # Update layout
    signals_fig.update_layout(title=f'Trading Signals for {company_name} ({stock_symbol})',
                              xaxis_title='Date', yaxis_title='Price', xaxis_rangeslider_visible=True)

    # Create the RSI plot
    rsi_fig = go.Figure()
    rsi_fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='blue')))
    rsi_fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Overbought", annotation_position="bottom right")
    rsi_fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Oversold", annotation_position="top right")
    # Highlight buy and sell signals
    rsi_fig.add_trace(go.Scatter(x=df[df['RSI_Signal'] == 1].index, y=df[df['RSI_Signal'] == 1]['RSI'],
                                 mode='markers', marker_symbol='triangle-up', marker_color='lime', marker_size=10, name='Buy Signal'))
    rsi_fig.add_trace(go.Scatter(x=df[df['RSI_Signal'] == -1].index, y=df[df['RSI_Signal'] == -1]['RSI'],
                                 mode='markers', marker_symbol='triangle-down', marker_color='crimson', marker_size=10, name='Sell Signal'))
    rsi_fig.update_layout(title=f'RSI and Trading Signals for {company_name} ({stock_symbol})',
                          xaxis_title='Date', yaxis_title='RSI', yaxis_range=[0, 100], xaxis_rangeslider_visible=True)

    df = calculate_macd(df)
    df['Signal'] = 0
    df.loc[(df['MACD'] > df['Signal_line']) & (df['MACD'].shift(1) < df['Signal_line'].shift(1)), 'Signal'] = 1   # Buy signal
    df.loc[(df['MACD'] < df['Signal_line']) & (df['MACD'].shift(1) > df['Signal_line'].shift(1)), 'Signal'] = -1  # Sell signal

    # Create the plot
    macd_fig = go.Figure()
    macd_fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue')))
    macd_fig.add_trace(go.Scatter(x=df.index, y=df['Signal_line'], name='Signal Line', line=dict(color='red')))

    # Highlight buy and sell signals
    macd_fig.add_trace(go.Scatter(x=df[df['Signal'] == 1].index, y=df[df['Signal'] == 1]['MACD'], 
                             mode='markers', marker_symbol='triangle-up', marker_color='lime', marker_size=10, name='Buy Signal'))
    macd_fig.add_trace(go.Scatter(x=df[df['Signal'] == -1].index, y=df[df['Signal'] == -1]['MACD'], 
                             mode='markers', marker_symbol='triangle-down', marker_color='crimson', marker_size=10, name='Sell Signal'))

    # Update layout
    macd_fig.update_layout(title=f'MACD and Trading Signals for {company_name} ({stock_symbol})', 
                      xaxis_title='Date', yaxis_title='MACD', xaxis_rangeslider_visible=True)
    
    df = calculate_stochastic_oscillator(df)
    df['Signal'] = 0
    df.loc[(df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1)), 'Signal'] = 1   # Buy signal
    df.loc[(df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1)), 'Signal'] = -1  # Sell signal

    stochastic_fig = go.Figure()
    stochastic_fig.add_trace(go.Scatter(x=df.index, y=df['%K'], name='%K', line=dict(color='blue')))
    stochastic_fig.add_trace(go.Scatter(x=df.index, y=df['%D'], name='%D', line=dict(color='red')))
    stochastic_fig.add_trace(go.Scatter(x=df[df['Signal'] == 1].index, y=df[df['Signal'] == 1]['%K'], 
                                    mode='markers', marker_symbol='triangle-up', marker_color='lime', marker_size=10, name='Buy Signal'))
    stochastic_fig.add_trace(go.Scatter(x=df[df['Signal'] == -1].index, y=df[df['Signal'] == -1]['%K'], 
                                    mode='markers', marker_symbol='triangle-down', marker_color='crimson', marker_size=10, name='Sell Signal'))
    stochastic_fig.update_layout(title=f'Stochastic Oscillator and Trading Signals for {company_name} ({stock_symbol})',
                             xaxis_title='Date', yaxis_title='Stochastic Oscillator', yaxis_range=[0, 100], xaxis_rangeslider_visible=True)
    
     # Create the Bollinger Bands plot
    bollinger_fig = go.Figure()
    bollinger_fig.add_trace(go.Scatter(x=df.index, y=df['Close'], name='Close Price', line=dict(color='blue')))
    bollinger_fig.add_trace(go.Scatter(x=df.index, y=df['Upper_Band'], name='Upper Band', line=dict(color='red')))
    bollinger_fig.add_trace(go.Scatter(x=df.index, y=df['Middle_Band'], name='Middle Band', line=dict(color='green')))
    bollinger_fig.add_trace(go.Scatter(x=df.index, y=df['Lower_Band'], name='Lower Band', line=dict(color='red')))
    bollinger_fig.add_trace(go.Scatter(x=df[df['Bollinger_Signal'] == 1].index, y=df[df['Bollinger_Signal'] == 1]['Close'], 
                                       mode='markers', marker_symbol='triangle-up', marker_color='lime', marker_size=10, name='Buy Signal'))
    bollinger_fig.add_trace(go.Scatter(x=df[df['Bollinger_Signal'] == -1].index, y=df[df['Bollinger_Signal'] == -1]['Close'], 
                                       mode='markers', marker_symbol='triangle-down', marker_color='crimson', marker_size=10, name='Sell Signal'))
    bollinger_fig.update_layout(title=f'Bollinger Bands and Trading Signals for {company_name} ({stock_symbol})',
                                xaxis_title='Date', yaxis_title='Price', xaxis_rangeslider_visible=True)
    
    return signals_fig, rsi_fig, macd_fig, stochastic_fig, bollinger_fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=False, port=8097)


Dash is running on http://127.0.0.1:8097/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8097/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Apr/2024 00:34:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 00:34:28] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_9_3m1682482245.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 00:34:28] "GET /_dash-component-suites/dash/deps/react@16.v2_9_3m1682482245.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 00:34:28] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_9_3m1682482245.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 00:34:28] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_9_3m1682482245.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 00:34:28] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_9_3m1682482244.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 00:34:28] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_9_2m1682482245.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2024 00:34:28] "GET /_dash-component-suites/d

In [3]:
import pandas as pd
import ipywidgets as widgets

us_stocks_df = pd.read_csv("ticker_correlation.csv")
us_stocks = us_stocks_df.iloc[:, 0].tolist()

# Initialize a dictionary to store stock data
stock_data = {}
company_names = {}

# Iterate over stock symbols
for symbol in us_stocks:
    ticker = yf.Ticker(symbol)
    data = ticker.history(start="2013-01-01", end=pd.Timestamp.today().strftime('%Y-%m-%d'))
    stock_data[symbol] = data
    company_names[symbol] = ticker.info.get('longName', 'N/A')

dropdown = widgets.Dropdown(options=us_stocks, description='Select Stock:')

def update_signals(stock_symbol):
    df = stock_data[stock_symbol].copy()

    # Calculate the required indicators
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()
    df['RSI'] = calculate_rsi(df)
    df = calculate_macd(df)
    df = calculate_bollinger_bands(df)
    df = calculate_stochastic_oscillator(df)

    # Create signal columns
    df['SMA_Signal'] = (df['SMA_50'] > df['SMA_200']).astype(int) - (df['SMA_50'] < df['SMA_200']).astype(int)
    df['EMA_Signal'] = (df['EMA_50'] > df['EMA_200']).astype(int) - (df['EMA_50'] < df['EMA_200']).astype(int)
    df['RSI_Signal'] = (df['RSI'] > 70).astype(int) - (df['RSI'] < 30).astype(int)
    df['Bollinger_Signal'] = (df['Close'] > df['Upper_Band']).astype(int) - (df['Close'] < df['Lower_Band']).astype(int)
    df['MACD_Signal'] = ((df['MACD'] > df['Signal_line']) & (df['MACD'].shift(1) <= df['Signal_line'].shift(1))).astype(int) - ((df['MACD'] < df['Signal_line']) & (df['MACD'].shift(1) >= df['Signal_line'].shift(1))).astype(int)
    df['Stochastic_Signal'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) <= df['%D'].shift(1))).astype(int) - ((df['%K'] < df['%D']) & (df['%K'].shift(1) >= df['%D'].shift(1))).astype(int)

    # Filter and create data frames for signals
    signals = {
        'SMA': df[df['SMA_Signal'] != 0][['Close', 'SMA_50', 'SMA_200', 'SMA_Signal']],
        'EMA': df[df['EMA_Signal'] != 0][['Close', 'EMA_50', 'EMA_200', 'EMA_Signal']],
        'RSI': df[df['RSI_Signal'] != 0][['Close', 'RSI', 'RSI_Signal']],
        'Bollinger': df[df['Bollinger_Signal'] != 0][['Close', 'Upper_Band', 'Middle_Band', 'Lower_Band', 'Bollinger_Signal']],
        'MACD': df[df['MACD_Signal'] != 0][['Close', 'MACD', 'Signal_line', 'MACD_Signal']],
        'Stochastic': df[df['Stochastic_Signal'] != 0][['Close', '%K', '%D', 'Stochastic_Signal']]
    }

    return signals

# Function to call when an item is selected in the dropdown menu
def dropdown_eventhandler(change):
    selected_stock = change.new
    signals = update_signals(selected_stock)  # Get the signals for the selected stock
    for indicator, signal_df in signals.items():
        print(f"Signals for {indicator} ({selected_stock}):")
        display(signal_df)

# Assign the event handler function to the dropdown's 'observe' method
dropdown.observe(dropdown_eventhandler, names='value')

# Display the dropdown menu
display(dropdown)

Dropdown(description='Select Stock:', options=('AAPL', 'ABNB', 'ADBE', 'ADM', 'ADT', 'ALTM', 'AMD', 'AMZN', 'A…